In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp

In [3]:
monthly_roi = pd.read_csv('roi_mat.csv')
monthly_roi = monthly_roi.rename(columns={'Unnamed: 0': 'Months'})
display(monthly_roi)
roi_firms = pd.read_csv('ROI_data.csv')
roi_firms = np.array(roi_firms.drop(['Platform'], axis=1))
roi_firms

,Months,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
1,February,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
2,March,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
3,April,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
4,May,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9
5,June,4.0,3.2,2.7,3.4,3.4,3.0,4.5,2.10,3.8,4.1
6,July,3.9,3.6,2.0,4.4,3.9,3.7,4.3,1.80,4.0,3.8
7,August,4.2,3.3,2.8,4.2,2.0,3.7,3.6,1.50,4.4,4.3
8,September,4.1,2.8,2.5,4.2,2.9,3.7,2.8,2.50,4.0,3.4
9,October,3.0,3.0,3.1,4.6,3.1,3.3,3.2,2.30,2.5,3.2


array([[0.031, 0.049, 0.024, 0.039, 0.016, 0.024, 0.046, 0.026, 0.033,
        0.044],
       [0.049, 0.023, 0.024, 0.039, 0.044, 0.046, 0.026, 0.019, 0.037,
        0.026]])

In [4]:
for rois in roi_firms:
    Mod = gp.Model()
    ModX = Mod.addMVar(10)

    obj = 0
    for i in range(len(rois)):
        obj += (1 + rois[i])*ModX[i]
    
    Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

    Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= 10)
    Mod.addConstr(ModX[0] + ModX[1] <= ModX[4] + ModX[9])
    Mod.addConstr(ModX[4] + ModX[5] + ModX[7] + ModX[6] + ModX[8] >= 2*(ModX[2] + ModX[3]))
    for i in range(10):
        Mod.addConstr(ModX[i] <= 3)

    Mod.Params.OutputFlag = 0
    Mod.optimize()
    print(Mod.objVal)
    print(ModX.X)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
10.456
[0. 3. 0. 1. 0. 0. 3. 0. 0. 3.]
10.456
[3. 0. 0. 1. 3. 3. 0. 0. 0. 0.]


In [5]:
# Add Question 5 in here!!!

In [6]:
Mod = gp.Model()
ModX = Mod.addMVar(10)

obj = 0
for i in range(len(roi_firms[0])):
    obj += (1 + roi_firms[0][i])*ModX[i]

Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= 10)
Mod.addConstr(ModX[0] + ModX[1] <= ModX[4] + ModX[9])
Mod.addConstr(ModX[4] + ModX[5] + ModX[7] + ModX[6] + ModX[8] >= 2*(ModX[2] + ModX[3]))
for i in range(10):
    Mod.addConstr(ModX[i] <= 3)

Mod.Params.OutputFlag = 0
Mod.optimize()
obj_val = Mod.objVal
ModX.X

print("Lower Bound", ModX.SAObjLow)
print("Upper Bound", ModX.SAObjUp)

Lower Bound [ -inf 1.039  -inf 1.033  -inf  -inf 1.039  -inf  -inf 1.029]
Upper Bound [1.049 1.062 1.039 1.046 1.029 1.039   inf 1.039 1.039   inf]


In [8]:
obj_vals = []
obj_vals.append(10)
allocations = []
for i, month in enumerate(monthly_roi['Months']):
    rois = np.array(monthly_roi.drop('Months', axis=1))
    rois = rois[i]
    Mod = gp.Model()
    ModX = Mod.addMVar(10)

    obj = 0
    for m in range(len(rois)):
        obj += (1 + (rois[m]/100))*ModX[m]

    Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

    if i==0:
        budget = obj_vals[i]
    else:
        budget = obj_vals[i-1] + (obj_vals[i] - obj_vals[i-1]) / 2

    Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= budget)
    Mod.addConstr(ModX[0] + ModX[1] <= ModX[4] + ModX[9])
    Mod.addConstr(ModX[4] + ModX[5] + ModX[7] + ModX[6] + ModX[8] >= 2*(ModX[2] + ModX[3]))
    for m in range(10):
        Mod.addConstr(ModX[m] <= 3)

    Mod.Params.OutputFlag = 0
    Mod.optimize()
    obj_val = Mod.objVal
    obj_vals.append(obj_val)
    allocations.append(list(ModX.X))
    print(ModX.X)
    print(obj_vals)

allocations = pd.DataFrame(allocations)
display(allocations)

[3.         0.         0.         1.33333333 0.         0.
 2.66666667 0.         0.         3.        ]
[10, 10.373]
[3.     0.     0.     2.3955 3.     0.     0.     0.     1.791  0.    ]
[10, 10.373, 10.592795999999998]
[0.       0.       0.       3.       0.       3.       1.482898 0.
 3.       0.      ]
[10, 10.373, 10.592795999999998, 10.900765225999999]
[0.         0.         0.         3.         0.         3.
 3.         0.         1.74678061 0.        ]
[10, 10.373, 10.592795999999998, 10.900765225999999, 11.166664715067999]
[2.03371497 0.         0.         0.         0.         0.
 3.         0.         3.         3.        ]
[10, 10.373, 10.592795999999998, 10.900765225999999, 11.166664715067999, 11.47389499450269]
[3.         0.         0.         0.         0.         0.
 3.         0.         2.32027985 3.        ]
[10, 10.373, 10.592795999999998, 10.900765225999999, 11.166664715067999, 11.47389499450269, 11.786450489267187]
[1.31508637 0.         0.         3.         

,0,1,2,3,4,5,6,7,8,9
0,3.000000,0.0,0.0,1.333333,0.000000,0.000000,2.666667,0.0,0.000000,3.00000
1,3.000000,0.0,0.0,2.395500,3.000000,0.000000,0.000000,0.0,1.791000,0.00000
2,0.000000,0.0,0.0,3.000000,0.000000,3.000000,1.482898,0.0,3.000000,0.00000
3,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,1.746781,0.00000
4,2.033715,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,3.000000,3.00000
5,3.000000,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,2.320280,3.00000
6,1.315086,0.0,0.0,3.000000,1.315086,0.000000,3.000000,0.0,3.000000,0.00000
7,3.000000,0.0,0.0,1.983367,0.000000,0.966733,0.000000,0.0,3.000000,3.00000
8,1.642480,0.0,0.0,3.000000,0.000000,3.000000,0.000000,0.0,3.000000,1.64248
9,0.000000,0.0,0.0,3.000000,0.610658,3.000000,3.000000,0.0,0.000000,3.00000
